## Import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Load in your data from kaggle.  
By working in a kaggle kernel, you can access the data directly from the competition, as well as make your submission without downloading your output file

In [ ]:
train = pd.read_csv('../input/climate-change-edsa2020-21/train.csv')
test = pd.read_csv('../input/climate-change-edsa2020-21/test.csv')

In [ ]:
print(train.shape)
print(test.shape)

display(train.head())
display(test.head())

percent_duplicates = round((1-(train['message'].nunique()/len(train['message'])))*100,2)
print('Duplicated tweets in train data:')
print(percent_duplicates,'%')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.sentiment.value_counts()

In [ ]:
print(train.isnull().sum())
print(test.isnull().sum())

## Splitting out the X variable from the target

In [ ]:
y = train['sentiment']
X = train['message']

In [ ]:
print(y)

In [ ]:
print(X)

## Pre-processing 

In [ ]:
def preprocessor(text):
    
    text = text.lower()
    text = re.sub(re.sub(r'\^[a-zA-Z]s+','', text))
    text = re.sub("\\s+(in|the|all|for|and|on)\\s+"," _connector_ ",text)
    
    words = re.split("\\s+", text)
    stemmed_words = [porter_stemmer.stem(word=word) for word in words]
    
    return ''.join(stemmed_words)

In [ ]:
def text_tokenizer(text):
    #space between special characters 
    text = re.sub("(\\W)","\\1", text)
                  
    #split whitespace
    return re.split("\\s+", text)

In [ ]:
print(train.isnull().sum())
print(test.isnull().sum())

## Turning text into something your model can read

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words="english")
X_vectorized = vectorizer.fit_transform(X)

In [ ]:
print(X_vectorized)

## Splitting the training data into a training and validation set

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=.3,shuffle=True, stratify=y, random_state=11)

## Training the model and evaluating using the validation set

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_val)

In [ ]:
lsvc = LinearSVC()
lsvc.fit(X_train, y_train)
lsvc_pred = lsvc.predict(X_val)

In [ ]:
from sklearn import metrics

print(metrics.classification_report(y_val, lsvc_pred))

## Getting our test set ready

In [ ]:
testx = test['message']
test_vect = vectorizer.transform(testx)

## Making predictions on the test set and adding a sentiment column to our original test df

In [ ]:
y_pred = lsvc.predict(test_vect)

In [ ]:
test['sentiment'] = y_pred

In [ ]:
test.head()

## Creating an output csv for submission

In [ ]:
test[['tweetid','sentiment']].to_csv('testsubmission.csv', index=False)